In [ ]:
!pip install huggingface_hub transformers protobuf sentencepiece accelerate peft

In [ ]:
!pip install ./diffusers

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
## no_grad -> inference_mode found no speedup.
## Adding torch compile 57 sec is base. 39 sec now.
## caching the model output -> 29 sec

In [1]:
import torch
# from diffusers import FluxPipeline
from pipeline_flux import FluxPipeline
from huggingface_hub import hf_hub_download
base_model_id = "black-forest-labs/FLUX.1-dev"
repo_name = "ByteDance/Hyper-SD"
# Take 8-steps lora as an example
ckpt_name = "Hyper-FLUX.1-dev-8steps-lora.safetensors"
# Load model, please fill in your access tokens since FLUX.1-dev repo is a gated model.
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
pipe.load_lora_weights(hf_hub_download(repo_name, ckpt_name))
pipe.fuse_lora(lora_scale=0.125)
pipe.to("cuda", dtype=torch.bfloat16)

FluxPipeline {
  "_class_name": "FluxPipeline",
  "_diffusers_version": "0.32.0.dev0",
  "_name_or_path": "black-forest-labs/FLUX.1-dev",
  "scheduler": [
    "diffusers",
    "FlowMatchEulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "T5TokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "FluxTransformer2DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [3]:
prompt = "A cat holding a sign that says hello world"

In [15]:
%%time
image = pipe(
    prompt,
    height=720,
    width=720,
    guidance_scale=3.5,
    num_inference_steps=8,
    max_sequence_length=512,
    generator=torch.Generator("cuda").manual_seed(0),
    compile=True,
    cache=True
).images[0]

  0%|          | 0/8 [00:00<?, ?it/s]

CPU times: user 4.03 s, sys: 491 ms, total: 4.52 s
Wall time: 3.85 s



| Measured on | Resolution, Steps | Time (sec)<sup>#</sup> |
|----------|----------|----------|
| Default Pipeline | 1024px, 28 | 33.1 |
| With `torch.compile` | 1024px, 28 | 25.1<sup>*</sup>|
| With `torch.compile` and model pred caching | 1024px, 28 | 18.1<sup>*</sup> |
| HyperSD LoRA With `torch.compile` | 1024px, 8 | 7.42<sup>*</sup> |
| HyperSD LoRA With `torch.compile` and model pred caching | 1024px, 8 | 5.68<sup>*</sup> |
| Default Pipeline | 512px, 28 | 9.91 |
| With `torch.compile` | 512px, 28 | 7.4<sup>*</sup> |
| With `torch.compile` and model pred caching | 512px, 28 | 5.61<sup>*</sup> |
| HyperSD LoRA With `torch.compile` | 512px, 8 | 2.25<sup>*</sup> |
| HyperSD LoRA With `torch.compile` and model pred caching | 512px, 8 | 1.75<sup>*</sup> |

<sup>*</sup> Compiliation time not included. <br>
<sup>#</sup> Wall time as measured with %%time magic. Not the tqdm measurement.